# Jet Compression using an Autoencoder

In [1]:
import sys
BIN = '../../'
sys.path.append(BIN)
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

from my_nn_modules import AE_big, get_data, fit

import my_matplotlib_style as ms
mpl.rc_file(BIN + 'my_matplotlib_rcparams')

## Loading and preparing the data

In [2]:
force_cpu = False

if force_cpu:
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
print('Using device:', device)

Using device: cpu


In [3]:
# Load data
train = pd.read_pickle(BIN + 'processed_data/train.pkl')
test = pd.read_pickle(BIN + 'processed_data/test.pkl')

train.head(10)

,pT,eta,phi,E
1367643,300.752869,0.118391,-1.460827,303.719818
1739612,79.819145,4.535507,-1.185196,3722.832764
546066,220.472305,1.314261,-0.943992,440.195190
213212,74.533775,0.010658,-0.424795,74.770332
150544,220.668121,2.432910,-2.308348,1266.681030
574837,159.841782,0.731125,-1.150598,207.133514
1171687,306.125305,0.702473,0.437615,385.713013
1608254,189.011673,-1.387443,0.490634,403.126709
1618830,194.996719,-1.022815,-1.560437,306.534515
637947,186.887146,-0.621282,-0.465523,226.002701


## Normalizing and splitting into train and test datasets

In [4]:
# Normalize
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
# Is this the right way to normalize? (only using train mean and std to normalize both train and test)
test = (test - train_mean) / train_std

train_x = train
test_x = test
train_y = train_x  # y = x since we are building and AE
test_y = test_x

train_ds = TensorDataset(torch.tensor(train_x.values), torch.tensor(train_y.values))
valid_ds = TensorDataset(torch.tensor(test_x.values), torch.tensor(test_y.values))

## Training

In [5]:
def mse_loss(input, target, size_average=None, reduce=None, reduction='mean'):
    # type: (Tensor, Tensor, Optional[bool], Optional[bool], str) -> Tensor
    r"""mse_loss(input, target, size_average=None, reduce=None, reduction='mean') -> Tensor

    Measures the element-wise mean squared error.

    See :class:`~torch.nn.MSELoss` for details.
    """
    if not (target.size() == input.size()):
        warnings.warn("Using a target size ({}) that is different to the input size ({}). "
                      "This will likely lead to incorrect results due to broadcasting. "
                      "Please ensure they have the same size.".format(target.size(), input.size()),
                      stacklevel=2)
    if size_average is not None or reduce is not None:
        reduction = _Reduction.legacy_get_string(size_average, reduce)

    ret = (input - target) ** 2
    factor = torch.full(ret.size(), 1)
    factor[:, 2] = 2.4
    ret = ret * factor
    if reduction != 'none':
        ret = torch.mean(ret) if reduction == 'mean' else torch.sum(ret)

    return ret

In [6]:
bs = 256  # batch size
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
loss_func = nn.MSELoss()
loss_func = mse_loss

In [7]:
class AE_2D_v200(nn.Module):
    def __init__(self, n_features=4):
        super(AE_2D_v200, self).__init__()
        self.en1 = nn.Linear(n_features, 200)
        self.en2 = nn.Linear(200, 200)
        self.en3 = nn.Linear(200, 200)
        self.en4 = nn.Linear(200, 2)
        self.de1 = nn.Linear(2, 200)
        self.de2 = nn.Linear(200, 200)
        self.de3 = nn.Linear(200, 200)
        self.de4 = nn.Linear(200, n_features)
        self.tanh = nn.Tanh()

    def encode(self, x):
        return self.en4(self.tanh(self.en3(self.tanh(self.en2(self.tanh(self.en1(x)))))))

    def decode(self, x):
        return self.de4(self.tanh(self.de3(self.tanh(self.de2(self.tanh(self.de1(self.tanh(x))))))))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

In [8]:
n_features = len(train.loc[0])
model = AE_2D_v200(n_features=n_features)

In [9]:
# Training
#epochs_list = [7, 5, 3, 2, 2]
epochs_list = [40, 40, 20, 20, 20]
lrs = [1e-3, 3e-4, 1e-4, 3e-5, 1e-5]
for ii, epochs in enumerate(epochs_list):
    print('Setting learning rate to %.1e' % lrs[ii])
    opt = optim.Adam(model.parameters(), lr=lrs[ii])
    #opt = optim.SGD(model.parameters(), lr=lrs[ii], momentum=0.9, nesterov=True)
    #opt = optim.RMSprop(model.parameters(), lr=lrs[ii], momentum=0.9)
    fit(epochs, model, loss_func, opt, train_dl, valid_dl, device)

Setting learning rate to 1.0e-03
Epoch 0: Validation loss = 0.13594044191633803 Time: 0:02:41.384107
Epoch 1: Validation loss = 0.13818707880988618 Time: 0:04:52.679778
Epoch 2: Validation loss = 0.06671269680361416 Time: 0:07:07.021613
Epoch 3: Validation loss = 0.05388856501750008 Time: 0:09:25.072279
Epoch 4: Validation loss = 0.047582941947826585 Time: 0:11:37.861837
Epoch 5: Validation loss = 0.04575329816088675 Time: 0:14:01.920800
Epoch 6: Validation loss = 0.039932629884080646 Time: 0:16:21.688385
Epoch 7: Validation loss = 0.03770655901307644 Time: 0:18:44.217058
Epoch 8: Validation loss = 0.041114305492305186 Time: 0:20:49.495613
Epoch 9: Validation loss = 0.055588596093631516 Time: 0:23:05.947859
Epoch 10: Validation loss = 0.03791319135102046 Time: 0:25:13.218617
Epoch 11: Validation loss = 0.03662667624310461 Time: 0:27:37.983169
Epoch 12: Validation loss = 0.03633540853273837 Time: 0:30:15.387924
Epoch 13: Validation loss = 0.03302988892699484 Time: 0:32:46.769869
Epoch 1

Epoch 18: Validation loss = 0.0359162006372659 Time: 0:38:54.441883
Epoch 19: Validation loss = 0.03608291271686125 Time: 0:40:56.362190
Setting learning rate to 1.0e-05
Epoch 0: Validation loss = 0.03583850114885109 Time: 0:01:53.491911
Epoch 1: Validation loss = 0.03587050324429887 Time: 0:03:52.896010
Epoch 2: Validation loss = 0.03633466362863317 Time: 0:05:53.436918
Epoch 3: Validation loss = 0.03576604600214862 Time: 0:07:44.995552
Epoch 4: Validation loss = 0.03593632968142009 Time: 0:09:43.387661
Epoch 5: Validation loss = 0.035751590640333994 Time: 0:11:50.642471
Epoch 6: Validation loss = 0.03564822089626921 Time: 0:13:57.604572
Epoch 7: Validation loss = 0.03618637544804768 Time: 0:15:58.290585
Epoch 8: Validation loss = 0.03589097626761049 Time: 0:18:01.920493
Epoch 9: Validation loss = 0.03586580859808621 Time: 0:20:04.404960
Epoch 10: Validation loss = 0.03591295706982002 Time: 0:22:06.968023
Epoch 11: Validation loss = 0.03584972170306964 Time: 0:24:08.169117
Epoch 12: V

In [10]:
# Training
#epochs_list = [7, 5, 3, 2, 2]
epochs_list = [40, 40, 20, 20, 20]
lrs = [1e-4, 3e-5, 1e-5, 3e-6, 1e-6]
for ii, epochs in enumerate(epochs_list):
    print('Setting learning rate to %.1e' % lrs[ii])
    opt = optim.Adam(model.parameters(), lr=lrs[ii])
    #opt = optim.SGD(model.parameters(), lr=lrs[ii], momentum=0.9, nesterov=True)
    #opt = optim.RMSprop(model.parameters(), lr=lrs[ii], momentum=0.9)
    fit(epochs, model, loss_func, opt, train_dl, valid_dl, device)

Setting learning rate to 1.0e-04
Epoch 0: Validation loss = 0.040168988096631546 Time: 0:02:02.738849
Epoch 1: Validation loss = 0.03904334575117154 Time: 0:04:04.531252
Epoch 2: Validation loss = 0.03799365354283852 Time: 0:06:06.952934
Epoch 3: Validation loss = 0.0394356475409341 Time: 0:08:10.684928
Epoch 4: Validation loss = 0.039405331949631923 Time: 0:10:18.420197
Epoch 5: Validation loss = 0.039100810284797796 Time: 0:12:15.440312
Epoch 6: Validation loss = 0.03847510450622685 Time: 0:14:10.671532
Epoch 7: Validation loss = 0.039538633310083046 Time: 0:16:15.618707
Epoch 8: Validation loss = 0.038955094921375784 Time: 0:18:10.332438
Epoch 9: Validation loss = 0.0387374168931995 Time: 0:20:03.436132
Epoch 10: Validation loss = 0.03812910713189027 Time: 0:21:57.489218
Epoch 11: Validation loss = 0.03810490061685725 Time: 0:23:48.617147
Epoch 12: Validation loss = 0.03875082779374368 Time: 0:25:40.478821
Epoch 13: Validation loss = 0.03862246022788678 Time: 0:27:32.332625
Epoch 14

KeyboardInterrupt: 

In [ ]:
# saving the model for later inference (if training is to be continued another saving method is recommended)
#save_path = './models/AE_2D_v4_bs256_loss0029.pt'
#torch.save(model.state_dict(), save_path)
# model_big = AE_big()
# model_big.load_state_dict(torch.load(save_path))
# model_big.eval()

## Evaluation

In [ ]:
model.eval()

In [ ]:
# Print a few tensors, still normalized
print('Comparing input and output:')
for ii in np.arange(100, 105):
    data = valid_ds.tensors[0][ii]
    pred = model(data)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

In [ ]:
# Print a few tensors, now not normalized
print('Comparing input and output:')
for ii in np.arange(100, 105):
    data = valid_ds.tensors[0][ii]
    pred = model(data).detach().numpy()
    pred = np.multiply(pred, train_std.values)
    pred = np.add(pred, train_mean.values)
    data = np.multiply(data, train_std.values)
    data = np.add(data, train_mean.values)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

In [ ]:
plt.close('all')
unit_list = ['[GeV]', '[rad]', '[rad]', '[GeV]']
variable_list = [r'$p_T$', r'$\eta$', r'$\phi$', r'$E$']
line_style = ['--', '-']
colors = ['orange', 'c']
markers = ['*', 's']


# Histograms
idxs = (0, 100000)  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
pred = model(data).detach().numpy()
pred = np.multiply(pred, train_std.values)
pred = np.add(pred, train_mean.values)
data = np.multiply(data, train_std.values)
data = np.add(data, train_mean.values)

alph = 0.8
n_bins = 50
for kk in np.arange(4):
    plt.figure(kk + 4)
    n_hist_data, bin_edges, _ = plt.hist(data[:, kk], color=colors[1], label='Input', alpha=1, bins=n_bins)
    n_hist_pred, _, _ = plt.hist(pred[:, kk], color=colors[0], label='Output', alpha=alph, bins=bin_edges)
    plt.suptitle(train.columns[kk])
    plt.xlabel(variable_list[kk] + ' ' + unit_list[kk])
    plt.ylabel('Number of events')
    plt.legend()
    ms.sciy()

In [ ]:
# Plot input on top of output
idxs = (0, 100)  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
pred = model(data).detach().numpy()
pred = np.multiply(pred, train_std.values)
pred = np.add(pred, train_mean.values)
data = np.multiply(data, train_std.values)
data = np.add(data, train_mean.values)

for kk in np.arange(4):
    plt.figure(kk)
    plt.plot(data[:, kk], color=colors[1], label='Input', linestyle=line_style[1], marker=markers[1])
    plt.plot(pred[:, kk], color=colors[0], label='Output', linestyle=line_style[0], marker=markers[0])
    plt.suptitle(train.columns[kk])
    plt.xlabel('Event')
    plt.ylabel(variable_list[kk] + ' ' + unit_list[kk])
    plt.legend()
    ms.sciy()

In [ ]:
# Plot input on top of output
idxs = (0, int(1e5))  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
latent = model.encode(data).detach().numpy()

In [ ]:
for ii in np.arange(latent.shape[1]):
    plt.figure()
    plt.hist(latent[:, ii], label='$z_%d$' % (ii + 1), color='m')
    plt.suptitle('Latent variable #%d' % (ii + 1))
    plt.legend()
    ms.sciy()

In [ ]:
mksz = 1

plt.figure()
plt.scatter(latent[:, 0], latent[:, 1], s=mksz)
plt.xlabel(r'$z_1$')
plt.ylabel(r'$z_2$')